In [12]:
!pip install 'git+https://github.com/facebookresearch/pytorch3d.git'

  Cloning https://github.com/facebookresearch/pytorch3d.git to /tmp/pip-req-build-0dwbu7lc
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/pytorch3d.git /tmp/pip-req-build-0dwbu7lc
  Resolved https://github.com/facebookresearch/pytorch3d.git to commit e13848265d9d57927fca99d13061e8fba8d468d0
  Preparing metadata (setup.py) ... done


In [13]:
!pip install numpy torch matplotlib

In [14]:
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from pytorch3d.loss import chamfer_distance
import time

In [15]:
# Funções utilitárias
def clear_folder(path):
    if os.path.exists(path):
        shutil.rmtree(path)
    os.mkdir(path)

def plotPCbatch(pcArray1, pcArray2, show=True, save=False, name=None, fig_count=9, sizex=12, sizey=3):
    pc1 = pcArray1[0:fig_count]
    pc2 = pcArray2[0:fig_count]
    fig = plt.figure(figsize=(sizex, sizey))
    
    for i in range(fig_count * 2):
        ax = fig.add_subplot(2, fig_count, i + 1, projection='3d')
        if i < fig_count:
            ax.scatter(pc1[i, :, 0], pc1[i, :, 2], pc1[i, :, 1], c='b', marker='.', alpha=0.8, s=8)
        else:
            ax.scatter(pc2[i - fig_count, :, 0], pc2[i - fig_count, :, 2], pc2[i - fig_count, :, 1], c='b', marker='.', alpha=0.8, s=8)
        ax.set_xlim3d(0.25, 0.75)
        ax.set_ylim3d(0.25, 0.75)
        ax.set_zlim3d(0.25, 0.75)
        plt.axis('off')
    
    plt.subplots_adjust(wspace=0, hspace=0)
    
    if save:
        fig.savefig(name + '.png')
        plt.close(fig)
    
    if show:
        plt.show()
    else:
        return fig

In [16]:
# Ler o dataset
class ReadDataset(Dataset):
    def __init__(self, source):
        self.data = torch.from_numpy(source).float()  # Converte o numpy array em um tensor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

def RandomSplit(datasets, train_set_percentage):
    lengths = [int(len(datasets) * train_set_percentage), len(datasets) - int(len(datasets) * train_set_percentage)]
    return random_split(datasets, lengths)

# Função para obter DataLoaders para treino e teste
def GetDataLoaders(npArray, batch_size, train_set_percentage=0.9, shuffle=True, num_workers=0, pin_memory=True):
    pc = ReadDataset(npArray)
    train_set, test_set = RandomSplit(pc, train_set_percentage)
    train_loader = DataLoader(train_set, shuffle=shuffle, num_workers=num_workers, batch_size=batch_size, pin_memory=pin_memory)  # Cria DataLoader para treino
    test_loader = DataLoader(test_set, shuffle=shuffle, num_workers=num_workers, batch_size=batch_size, pin_memory=pin_memory)  # Cria DataLoader para teste
    return train_loader, test_loader 

In [17]:
# Definição do modelo PointCloudAE
class PointCloudAE(nn.Module):
    def __init__(self, point_size, latent_size):
        super(PointCloudAE, self).__init__()
        self.latent_size = latent_size
        self.point_size = point_size
        self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, self.latent_size, 1)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(self.latent_size)
        self.dec1 = nn.Linear(self.latent_size, 256)
        self.dec2 = nn.Linear(256, 256)
        self.dec3 = nn.Linear(256, self.point_size * 3)

    def encoder(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, self.latent_size)
        return x

    def decoder(self, x):
        x = F.relu(self.dec1(x))
        x = F.relu(self.dec2(x))
        x = self.dec3(x)
        return x.view(-1, self.point_size, 3)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [18]:
def train_epoch(net, train_loader, optimizer, device):
    epoch_loss = 0
    for data in train_loader:
        optimizer.zero_grad()
        data = data.to(device)
        output = net(data.permute(0, 2, 1))
        loss, _ = chamfer_distance(data, output)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(train_loader)

def test_epoch(net, test_loader, device):
    epoch_loss = 0
    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)
            output = net(data.permute(0, 2, 1))
            loss, _ = chamfer_distance(data, output)
            epoch_loss += loss.item()
    return epoch_loss / len(test_loader)

def test_batch(net, data, device):
    with torch.no_grad():
        data = data.to(device)
        output = net(data.permute(0, 2, 1))
        loss, _ = chamfer_distance(data, output)
    return loss.item(), output.cpu()

In [19]:
# Parâmetros principais
batch_size = 32
latent_size = 128
output_folder = "output/"
save_results = True
use_GPU = True

pc_array = np.load("/kaggle/input/point-cloud-chair/chair_set.npy")
train_loader, test_loader = GetDataLoaders(pc_array, batch_size)

In [20]:
point_size = len(train_loader.dataset[0])
net = PointCloudAE(point_size, latent_size)

device = torch.device("cuda:0" if use_GPU and torch.cuda.is_available() else "cpu")
if torch.cuda.device_count() > 1:
    net = torch.nn.DataParallel(net)

net = net.to(device)
optimizer = optim.Adam(net.parameters(), lr=0.0005)

In [21]:
if save_results:
    clear_folder(output_folder)

train_loss_list = []
test_loss_list = []

In [22]:
# Treinamento
for epoch in range(1001):
    start_time = time.time()
    
    train_loss = train_epoch(net, train_loader, optimizer, device)
    test_loss = test_epoch(net, test_loader, device)
    
    train_loss_list.append(train_loss)
    test_loss_list.append(test_loss)
    
    epoch_time = time.time() - start_time
    print(f"Epoch {epoch}: Train Loss = {train_loss}, Test Loss = {test_loss}, Time = {epoch_time}s")
    
    if save_results:
        with open(output_folder + "results.txt", "a") as file:
            file.write(f"Epoch {epoch}: Train Loss = {train_loss}, Test Loss = {test_loss}, Time = {epoch_time}s\n")
        
        plt.plot(train_loss_list, label="Train")
        plt.plot(test_loss_list, label="Test")
        plt.legend()
        plt.savefig(output_folder + "loss.png")
        plt.close()
        
        if epoch % 50 == 0:
            test_samples = next(iter(test_loader))
            loss, test_output = test_batch(net, test_samples, device)
            plotPCbatch(test_samples.cpu().numpy(), test_output.numpy(), save=True, name=output_folder + f"epoch_{epoch}")


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):


Epoch 0: Train Loss = 0.027925203609245143, Test Loss = 0.002937798291289558, Time = 3.686427116394043s
Epoch 1: Train Loss = 0.002823838801562505, Test Loss = 0.002334355318453163, Time = 1.6069164276123047s
Epoch 2: Train Loss = 0.002352531202014465, Test Loss = 0.002313179507230719, Time = 1.5773401260375977s
Epoch 3: Train Loss = 0.0021189504614943324, Test Loss = 0.0019476922364750255, Time = 1.6120355129241943s
Epoch 4: Train Loss = 0.002063045044262665, Test Loss = 0.0018852569143443059, Time = 1.6636314392089844s
Epoch 5: Train Loss = 0.0019106310781284745, Test Loss = 0.0018246012235370774, Time = 1.848430871963501s
Epoch 6: Train Loss = 0.0018118608925263135, Test Loss = 0.0016468933996899675, Time = 1.6656241416931152s
Epoch 7: Train Loss = 0.0017211372592873028, Test Loss = 0.0015758723020553589, Time = 1.606431484222412s
Epoch 8: Train Loss = 0.0016480470888914083, Test Loss = 0.0017136239912360907, Time = 1.6076412200927734s
Epoch 9: Train Loss = 0.0016167536970965986, Te

In [ ]:
# Célula 7: Extração da representação latente para novos dados (Iza.L)
# Carregar novo dataset e extrair representação latente após o treinamento (Iza.L)

#new_data_np = np.load("seu_novo_dataset.npy")  # Substituir pelo seu novo dataset
#new_data = torch.from_numpy(new_data_np).float().to(device)

#with torch.no_grad():
    # Extrai apenas a representação latente da nova nuvem de pontos
    #latent_representation = net(new_data.permute(0, 2, 1), return_latent=True)

#print("Representação latente obtida:", latent_representation.shape)
